In [ ]:
import pandas as pd
from pathlib import Path
from src.load_scripts import get_code, parse_code, clean_code

data_path = Path('data/')

item = pd.read_csv(data_path / 'umimeprogramovatcz-ipython_item.csv', sep=';')

In [ ]:
parse_code(item['democode'][2], True)

In [ ]:
item['democode'] = item['democode'].apply(get_code).apply(parse_code).apply(clean_code)

In [ ]:
item['democode'][2]

In [ ]:
item.head()

In [ ]:
item['democode'][0]